Set up env

In [1]:
import carla
import numpy as np
from src.simulator import Simulator
from src.encoder import EncoderResnet18
import torch
from src.agent import NCPAgent
from src.model import Model, SequenceLearner

simulator = Simulator(world_name='Town04_opt')


In [2]:
simulator.world.get_spectator().set_transform(
    carla.Transform(
        location=carla.Location(x=398.7934265136719,
                                y=-56.03200912475586,
                                z=3.37939715385437)))

simulator.spawn_car_with_camera(
    rel_coordinates=carla.Location(x=1.2, z=1.9) # camera coords
)
vehicle = simulator.get_vehicle()


output_size = 2
units = 512

ncp = Model(output_size, units)

agent = NCPAgent(simulator, ncp)

next_waypoint  = [simulator.world.get_map().get_waypoint(vehicle.get_location(),
                                                    project_to_road=True,
                                                    lane_type=(carla.LaneType.Driving))]

waypoints = []
for _ in range(150):
    # print(next_waypoint[-1].transform)
    waypoints.append(next_waypoint[-1])
    # simulator.world.get_spectator().set_transform(next_waypoint[-1].transform)
    next_waypoint = next_waypoint[-1].next(15)

dest = waypoints[3].transform.location
agent.set_destination(dest)
agent.set_target_speed(10)

alloc!
Camera stream started


In [3]:
seq_learner = SequenceLearner(ncp, 0.005)

In [ ]:
simulator.world.get_spectator().set_transform(waypoints[1].transform)

In [4]:
import time
vehicle.apply_control(carla.VehicleControl(throttle=0.6, steer = 0.0))
waypoint = waypoints[1]

while True:
    time.sleep(1)
    if (np.abs(vehicle.get_location().x - waypoint.transform.location.x)) < 1 and \
        (np.abs(vehicle.get_location().y - waypoint.transform.location.y)) < 1:
        waypoint = waypoint.next(15)[0]
    # print(waypoint.transform.location)
    control, collided, true_angle, raw_data = agent.run_step(vehicle.get_location(), waypoint.transform.location)
    # print(seq_learner.training_step(raw_data, true_angle))
    if agent.done():
        print("The target has been reached, stopping the simulation")
        break
    if collided == True:
        vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))
        break

    vehicle.apply_control(carla.VehicleControl(throttle = 0.55, brake=0.0, steer = true_angle.item()))
        # print("Collision detected. Abort")
        # break
    # vehicle.apply_control(control)
    # time.sleep(0.5)

tensor([-0.1173, -0.0367], grad_fn=<SelectBackward0>)
Angle_cos = 0.9999924240024108
Angle in rad = 0.0038925588383827256
Angle in deg = 0.22302719294567652
Steering angle = 0.0031861027563668074
tensor([-0.1172, -0.0367], grad_fn=<SelectBackward0>)
Angle_cos = 0.9999963021984737
Angle in rad = 0.002719486644671292
Angle in deg = 0.15581510718185843
Steering angle = 0.0022259301025979775
tensor([-0.1171, -0.0366], grad_fn=<SelectBackward0>)
Angle_cos = 0.9999903806073599
Angle in rad = 0.004386207487628019
Angle in deg = 0.25131117710976575
Steering angle = 0.0035901596729966535
tensor([-0.1172, -0.0365], grad_fn=<SelectBackward0>)
Angle_cos = 0.9999184125035606
Angle in rad = 0.012774083597929895
Angle in deg = 0.7319010773086727
Steering angle = 0.010455729675838181
tensor([-0.1173, -0.0371], grad_fn=<SelectBackward0>)
Angle_cos = 0.9865031619477628
Angle in rad = 0.16448287798226544
Angle in deg = 9.424174710549103
Steering angle = 0.13463106729355862
tensor([-0.1172, -0.0366], grad

In [ ]:
simulator.destroy_all()

In [ ]:
# import torch
# import torch.nn as nn
# import torchvision
# from torchvision import models, transforms, utils
# from torch.autograd import Variable
# import numpy as np
# import matplotlib.pyplot as plt
# import scipy.misc
# from PIL import Image
# import json

# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=0., std=1.)
# ])

# image = Image.open(str('out/636.png'))
# plt.imshow(image)

In [ ]:
# from src.encoder import EncoderResnet18
# enc = EncoderResnet18()
# model = enc.model

In [ ]:
# # we will save the conv layer weights in this list
# model_weights =[]
# #we will save the 49 conv layers in this list
# conv_layers = []
# # get all the model children as list
# model_children = list(model.children())
# #counter to keep count of the conv layers
# counter = 0
# #append all the conv layers and their respective wights to the list
# for i in range(len(model_children)):
#     if type(model_children[i]) == nn.Conv2d:
#         counter+=1
#         model_weights.append(model_children[i].weight)
#         conv_layers.append(model_children[i])
#     elif type(model_children[i]) == nn.Sequential:
#         for j in range(len(model_children[i])):
#             for child in model_children[i][j].children():
#                 if type(child) == nn.Conv2d:
#                     counter+=1
#                     model_weights.append(child.weight)
#                     conv_layers.append(child)
# print(f"Total convolution layers: {counter}")
# print(f"{conv_layers}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# image = transform(image)
# image = image[:3,:,:]
# print(f"Image shape before: {image.shape}")
# image = image.unsqueeze(0)
# print(f"Image shape after: {image.shape}")
# image = image.to(device)

In [ ]:
# outputs = []
# names = []
# for layer in conv_layers[0:]:
#     image = layer(image)
#     outputs.append(image)
#     names.append(str(layer))
# print(len(outputs))
# # print feature_maps
# for feature_map in outputs:
#     print(feature_map.shape)

In [ ]:
# processed = []
# for feature_map in outputs:
#     feature_map = feature_map.squeeze(0)
#     gray_scale = torch.sum(feature_map,0)
#     gray_scale = gray_scale / feature_map.shape[0]
#     processed.append(gray_scale.data.cpu().numpy())
# for fm in processed:
#     print(fm.shape)


In [ ]:
# fig = plt.figure(figsize=(30, 50))
# for i in range(len(processed)):
#     a = fig.add_subplot(5, 4, i+1)
#     imgplot = plt.imshow(processed[i])
#     a.axis("off")
#     a.set_title(names[i].split('(')[0], fontsize=30)
# plt.savefig(str('feature_maps.jpg'), bbox_inches='tight')


In [ ]:
# import torch
# import torch.nn as nn
# import pytorch_lightning as pl
# import torch.utils.data as data

# from src.model import SequenceLearner, NCP

# ncp = NCP(image_features.shape[1], output_size, units)
# ncp_model = ncp.get_ncp()
# learner = SequenceLearner(ncp_model, lr=0.01)
# trainer = pl.Trainer(
#     logger=pl.loggers.CSVLogger("log"),
#     max_epochs=5,
#     gradient_clip_val=1, #clip grad for training stabilizing
# )

Agent test

In [ ]:
import glob, os, sys
# sys.path.append(glob.glob('CARLA_SIM/PythonAPI/carla/'))
sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent
# from agents.navigation.behavior_agent import BehaviorAgent

In [ ]:
simulator.spawn_car_with_camera()

In [ ]:
vehicle = simulator.get_vehicle()

In [ ]:
# agent = BehaviorAgent(vehicle, behavior='aggressive')
agent = BasicAgent(vehicle)
dest = simulator.spawn_points[50].location
agent.set_destination(dest)
agent.follow_speed_limits(False)
agent.set_target_speed(90)

In [ ]:
import time
while True:
    if agent.done():
        print("The target has been reached, stopping the simulation")
        break
    # if collided == True:
    #     vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))
    #     print("Collision detected. Abort")
    #     break

    vehicle.apply_control(agent.run_step())
    # time.sleep(0.5)

In [ ]:
simulator.destroy_all()